In [ ]:
import sys
import glob
  
from genophenocorr import *
from genophenocorr import compare
from genophenocorr import compare_func as CF

In [ ]:
allPatients = AllPatients('../phenopackets/PlatzerData/*.json')

In [ ]:
[p.describe() for p in allPatients.all_patients.values()]

In [ ]:
allPatients.count_vars_per_feature()

In [ ]:
for var in allPatients.all_variants:
    print(var.variant_type)
    print(var.variant.short_description)
    print(var.variant.is_indel)
    print(var.variant.effects()[0])

In [ ]:
for typ in allPatients.all_var_types:
    print(f'Comparing all in {typ} vs out of {typ}')
    compare.RunStats(allPatients, CF.is_var_type, CF.is_not_var_type, typ, typ)
    for typ2 in allPatients.all_var_types:
        if typ2 == typ:
            continue
        print(f'Comparing all in {typ} vs all in {typ2}')
        compare.RunStats(allPatients, CF.is_var_type, CF.is_var_type, typ, typ2)

In [ ]:
for var in allPatients.all_variants:
    print(f'Comparing affecting {var.variant_string} vs all not')
    compare.RunStats(allPatients, CF.is_var_match, CF.is_not_var_match, var, var)

In [ ]:
compare.RunStats(allPatients,CF.in_feature, CF.not_in_feature,'Domain','Domain')

In [ ]:
Protein.findProtein('ENSP00000501096')

In [ ]:
compare.RunStats(allPatients, CF.is_var_type, CF.is_not_var_type, 'missense', 'missense')
